# Steam Games Keywords Linker
This is a sample jupyter notebook that shows how the project works. We use the Steam Games Store Dataset from Kaggle to extract keywords from the games store short description. This allows us to link games based on their keywords. The result of this notebook is a .ttl file that you can use to create queries, in this project we use Sparql.

In [1]:
# if on colab, you can install the requirements with the following command
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.2 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39126 sha256=3616f140c58618fc39b058f332d1e12ebf0df9047081503dcf1b3250e98e166f
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9fc54ba3e6bd4234b9b7dbbeb1f2b388beb74e7c1ecba3bf640c6d51d078fde4
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [3]:
# download and unzip the dataset (check repo readme for attribution)
!wget https://github.com/iPolarisu/keyword-doc-linker/raw/main/steam_store.zip
!unzip steam_store.zip

--2023-11-23 22:11:13--  https://github.com/iPolarisu/keyword-doc-linker/raw/main/steam_store.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/iPolarisu/keyword-doc-linker/main/steam_store.zip [following]
--2023-11-23 22:11:13--  https://raw.githubusercontent.com/iPolarisu/keyword-doc-linker/main/steam_store.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36916941 (35M) [application/zip]
Saving to: ‘steam_store.zip’

steam_store.zip     100%[===================>]  35.21M   203MB/s    in 0.2s    

2023-11-23 22:11:14 (203 MB/s) - ‘steam_store.zip’ saved [36916941/36916941]

--2023-11-23 22

## Process the raw dataset

In this section we will process the raw steam data, by doing this we will be able to add each game's present keyphrases.

In [4]:
# standard library imports
import itertools
import re

# third-party imports
import numpy as np
import pandas as pd


def remove_non_english(df):
    # keep only rows marked as supporting english
    df = df[df['english'] == 1].copy()

    # keep rows which don't contain 3 or more non-ascii characters in succession
    df = df[~df['name'].str.contains('[^\u0001-\u007F]{3,}')]

    # remove english column, now redundant
    df = df.drop('english', axis=1)

    return df

def calc_rating(row):
    """Calculate rating score based on SteamDB method."""
    import math

    pos = row['positive_ratings']
    neg = row['negative_ratings']

    total_reviews = pos + neg
    average = pos / total_reviews

    # pulls score towards 50, pulls more strongly for games with few reviews
    score = average - (average*0.5) * 2**(-math.log10(total_reviews + 1))

    return score * 100

def pre_process(df):
    # keep english only
    df = remove_non_english(df)

    # keep windows only, and remove platforms column
    df = df[df['platforms'].str.contains('windows')].drop('platforms', axis=1).copy()

    # keep lower bound of owners column, as integer
    df['owners'] = df['owners'].str.split('-').apply(lambda x: x[0]).astype(int)

    # calculate rating, as well as simple ratio for comparison
    df['total_ratings'] = df['positive_ratings'] + df['negative_ratings']
    df['rating_ratio'] = df['positive_ratings'] / df['total_ratings']
    df['rating'] = df.apply(calc_rating, axis=1)

    # convert release_date to datetime type and create separate column for release_year
    df['release_date'] = df['release_date'].astype('datetime64[ns]')
    df['release_year'] = df['release_date'].apply(lambda x: x.year)

    return df

In [5]:
# raw dataset
df_steam = pd.read_csv('steam.csv')
df_steam.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [7]:
# processed dataset
df_steam = pre_process(df_steam)
df_steam.head()

,appid,name,release_date,developer,publisher,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,total_ratings,rating_ratio,rating,release_year
0,10,Counter-Strike,2000-11-01,Valve,Valve,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000,7.19,127873,0.973888,95.975678,2000
1,20,Team Fortress Classic,1999-04-01,Valve,Valve,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000,3.99,3951,0.839787,80.508259,1999
2,30,Day of Defeat,2003-05-01,Valve,Valve,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000,3.99,3814,0.895648,85.823915,2003
3,40,Deathmatch Classic,2001-06-01,Valve,Valve,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000,3.99,1540,0.826623,78.126542,2001
4,50,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000,3.99,5538,0.947996,91.260480,1999


In [10]:
# extraction of games with the highest owners
df_steam_key = df_steam.nlargest(5000, 'owners')
# select columns
df_steam_key = df_steam_key[['appid','name','rating','owners','median_playtime']]

df_steam_description = pd.read_csv('steam_description_data.csv')
# add the detailed_description column of steam_description to the df_steam_key (based on appid=steam_appid)
df_steam_key = df_steam_key.merge(df_steam_description[['steam_appid', 'short_description']], left_on='appid', right_on='steam_appid', how='left')
df_steam_key = df_steam_key.drop('steam_appid', axis=1)

df_steam_key.head()

,appid,name,rating,owners,median_playtime,short_description
0,570,Dota 2,85.201281,100000000,801,"Every day, millions of players worldwide enter..."
1,730,Counter-Strike: Global Offensive,86.310312,50000000,6502,Counter-Strike: Global Offensive (CS: GO) expa...
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,50.066901,50000000,12434,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...
3,440,Team Fortress 2,92.933233,20000000,623,Nine distinct classes provide a broad range of...
4,230410,Warframe,90.695302,20000000,394,Warframe is a cooperative free-to-play third p...


In [11]:
from keybert import KeyBERT
kw_model = KeyBERT()

# Extract keyphrases:
# Define a function to extract keyphrases from the descriptions
def extract_top_keyphrases(description):
    keywords = kw_model.extract_keywords(description, keyphrase_ngram_range=(2, 4), stop_words='english', top_n=8)
    return ', '.join([keyword[0] for keyword in keywords])

# Apply the function to the 'detailed_description' column to create a new column 'top_keyphrases'
df_steam_key['top_keyphrases'] = df_steam_key['short_description'].apply(extract_top_keyphrases)
# Now, df_steam_key contains a new 'top_keyphrases' column with the top 5 keyphrases for each game description.

df_steam_key.head()

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

,appid,name,rating,owners,median_playtime,short_description,top_keyphrases
0,570,Dota 2,85.201281,100000000,801,"Every day, millions of players worldwide enter...","battle dota heroes matter, battle dota heroes,..."
1,730,Counter-Strike: Global Offensive,86.310312,50000000,6502,Counter-Strike: Global Offensive (CS: GO) expa...,"counter strike global offensive, strike global..."
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,50.066901,50000000,12434,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"playerunknown battlegrounds battle royale, pla..."
3,440,Team Fortress 2,92.933233,20000000,623,Nine distinct classes provide a broad range of...,"tactical abilities, classes provide broad rang..."
4,230410,Warframe,90.695302,20000000,394,Warframe is a cooperative free-to-play third p...,"warframe cooperative free play, warframe coope..."


In [12]:
# save csv
df_steam_key.to_csv('steam_keyphrases.csv', index=False)

## TTL Parser
This second half of the notebook focuses on parsing the generated csv file that contains the keyphrases while creating a ttl file we can query, we use the csv instead of the dataframe if you want to skip the first half.

In [15]:
!wget https://raw.githubusercontent.com/iPolarisu/keyword-doc-linker/main/data/steam_keyphrases.csv

--2023-11-23 22:47:32--  https://raw.githubusercontent.com/iPolarisu/keyword-doc-linker/main/data/steam_keyphrases.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2375019 (2.3M) [text/plain]
Saving to: ‘steam_keyphrases.csv’

steam_keyphrases.cs 100%[===================>]   2.26M  --.-KB/s    in 0.07s   

2023-11-23 22:47:33 (31.7 MB/s) - ‘steam_keyphrases.csv’ saved [2375019/2375019]



In [16]:
df = pd.read_csv("steam_keyphrases.csv", encoding="utf8")

In [17]:
df.head()

,appid,name,rating,owners,median_playtime,short_description,top_keyphrases
0,570,Dota 2,85.201281,100000000,801,"Every day, millions of players worldwide enter...","battle dota heroes matter, battle dota heroes,..."
1,730,Counter-Strike: Global Offensive,86.310312,50000000,6502,Counter-Strike: Global Offensive (CS: GO) expa...,"counter strike global offensive, strike global..."
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,50.066901,50000000,12434,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"playerunknown battlegrounds battle royale, pla..."
3,440,Team Fortress 2,92.933233,20000000,623,Nine distinct classes provide a broad range of...,"tactical abilities, classes provide broad rang..."
4,230410,Warframe,90.695302,20000000,394,Warframe is a cooperative free-to-play third p...,"warframe cooperative free play, warframe coope..."


In [20]:
df['top_keyphrases'] = df.apply(lambda row: str(row[3]).split(',') if not pd.isnull(row[3]) else [], axis=1)
df['developer'] = df.apply(lambda row: str(row[5]).split(';') if not pd.isnull(row[5]) else [], axis=1)

In [ ]:
df.groupby(['name']).count()

,appid,rating,detailed_description,top_keyphrases
name,,,,
A Story About My Uncle,5,5,5,5
APB Reloaded,5,5,5,5
ARK: Survival Evolved,5,5,5,5
Age of Empires II HD,5,5,5,5
Alien Swarm,5,5,5,5
...,...,...,...,...
Warface,5,5,5,5
Warframe,5,5,5,5
"Warhammer 40,000: Dawn of War II",5,5,5,5


In [21]:
df.head()

,appid,name,rating,owners,median_playtime,short_description,top_keyphrases,developer
0,570,Dota 2,85.201281,100000000,801,"Every day, millions of players worldwide enter...",[100000000],"[Every day, millions of players worldwide ente..."
1,730,Counter-Strike: Global Offensive,86.310312,50000000,6502,Counter-Strike: Global Offensive (CS: GO) expa...,[50000000],[Counter-Strike: Global Offensive (CS: GO) exp...
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,50.066901,50000000,12434,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,[50000000],[PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roy...
3,440,Team Fortress 2,92.933233,20000000,623,Nine distinct classes provide a broad range of...,[20000000],[Nine distinct classes provide a broad range o...
4,230410,Warframe,90.695302,20000000,394,Warframe is a cooperative free-to-play third p...,[20000000],[Warframe is a cooperative free-to-play third ...


In [22]:
from urllib.parse import quote, unquote

def keep_alphanumeric(input_string):
    return ''.join(char for char in input_string if char.isalnum() or char == ' ')


def encode_as_iri(input_string):
    return quote(input_string, safe='')

def to_rdf(df):
    prelude = """@prefix : <http://ex.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
"""
    retstr = ""
    for _,row in df.iterrows():
        name = row["name"]
        name = keep_alphanumeric(name)
        retstr += f':{row["appid"]} rdfs:label "{name}"; :rating "{row["rating"]}"^^xsd:float; :med_play "{row["median_playtime"]}"^^xsd:int; :owners "{row["owners"]}"^^xsd:int;'
        s= ""
        for i,kp in enumerate(row['top_keyphrases']):
            kp2add = kp.strip().replace(" ", "_")
            kp2add = encode_as_iri(kp2add)
            if i==4:
                s+= f' :hasKP :{kp2add};'
                break
            s += f' :hasKP :{kp2add} ;'
        retstr += s
        s=""
        for i,dev in enumerate(row['developer']):
            dev2add = dev.strip().replace(" ", "_").replace(".","")
            dev2add = encode_as_iri(dev2add)
            if i == (len(row['developer']) - 1):
                s+= f' :dev :{dev2add}'
                break
            s += f' :dev :{dev2add} ;'
        retstr += s
        retstr += " .\n"
    return prelude+retstr

In [23]:
with open("steam_keyphrases.ttl", "w", encoding="utf-8") as f:
    f.write(to_rdf(df))